In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('RDDops').getOrCreate()

print('Spark Object is Created')

Spark Object is Created


In [13]:
# Transformation Low Level 

ord = spark.sparkContext.textFile('RetailDB SalesData/Orders/part-00000')
orditem = spark.sparkContext.textFile('RetailDB SalesData/Order_items/part-00000')

print('Files loaded')

Files loaded


In [14]:
# to view the loaded files

print('To view the data in Order File: ')
for o in ord.take(10):
    print(o)
    
print('-------------------------')
print('-------------------------')
print('-------------------------')

print('To view the data in OrderItem File: ')
for oi in orditem.take(10):
    print(oi)

To view the data in Order File: 
1,2013-07-25 00:00:00.0,11599,CLOSED
2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
3,2013-07-25 00:00:00.0,12111,COMPLETE
4,2013-07-25 00:00:00.0,8827,CLOSED
5,2013-07-25 00:00:00.0,11318,COMPLETE
6,2013-07-25 00:00:00.0,7130,COMPLETE
7,2013-07-25 00:00:00.0,4530,COMPLETE
8,2013-07-25 00:00:00.0,2911,PROCESSING
9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT
10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT
-------------------------
-------------------------
-------------------------
To view the data in OrderItem File: 
1,1,957,1,299.98,299.98
2,2,1073,1,199.99,199.99
3,2,502,5,250.0,50.0
4,2,403,1,129.99,129.99
5,4,897,2,49.98,24.99
6,4,365,5,299.95,59.99
7,4,502,3,150.0,50.0
8,4,1014,4,199.92,49.98
9,5,957,1,299.98,299.98
10,5,365,5,299.95,59.99


In [15]:
# show all ids in order file
ord_id = ord.map(lambda x : x.split(',')[0])

print('The Order Id in Order File:')
print('-----------------------------')
for id in ord_id.take(10):
    print(id)

The Order Id in Order File:
-----------------------------
1
2
3
4
5
6
7
8
9
10


In [22]:
# show all the orders and its status

ord_status = ord.map(lambda x : (x.split(',')[0],x.split(',')[3]))

print('The Orderid and Status as Follows: ')
print('----------------------------------')
for s in ord_status.take(10):
    print(s)


The Orderid and Status as Follows: 
----------------------------------
('1', 'CLOSED')
('2', 'PENDING_PAYMENT')
('3', 'COMPLETE')
('4', 'CLOSED')
('5', 'COMPLETE')
('6', 'COMPLETE')
('7', 'COMPLETE')
('8', 'PROCESSING')
('9', 'PENDING_PAYMENT')
('10', 'PENDING_PAYMENT')


In [26]:
# show all the orders and its status in combined way

ord_statusc = ord.map(lambda x : (x.split(',')[0]+'#'+x.split(',')[3]))

print('The Orderid and Status as Follows: ')
print('----------------------------------')
for s in ord_statusc.take(10):
    print(s)

The Orderid and Status as Follows: 
----------------------------------
1#CLOSED
2#PENDING_PAYMENT
3#COMPLETE
4#CLOSED
5#COMPLETE
6#COMPLETE
7#COMPLETE
8#PROCESSING
9#PENDING_PAYMENT
10#PENDING_PAYMENT


In [30]:
# convert order date into the yyyy:mm:dd
ord_date = ord.map(lambda d : d.split(',')[1].split(' ')[0].replace('-','/'))

print('The Date in the Format replaced: ')
print('---------------------------------')
for d in ord_date.take(10):
    print(d)


The Date in the Format replaced: 
---------------------------------
2013/07/25
2013/07/25
2013/07/25
2013/07/25
2013/07/25
2013/07/25
2013/07/25
2013/07/25
2013/07/25
2013/07/25


In [34]:
# making id as key and value as whole record

ord_kv = ord.map(lambda k : (k.split(',')[0],k))

print('The key Value pairs:')
print('---------------------')
for kv in ord_kv.take(10):
    print(kv)

The key Value pairs:
---------------------
('1', '1,2013-07-25 00:00:00.0,11599,CLOSED')
('2', '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT')
('3', '3,2013-07-25 00:00:00.0,12111,COMPLETE')
('4', '4,2013-07-25 00:00:00.0,8827,CLOSED')
('5', '5,2013-07-25 00:00:00.0,11318,COMPLETE')
('6', '6,2013-07-25 00:00:00.0,7130,COMPLETE')
('7', '7,2013-07-25 00:00:00.0,4530,COMPLETE')
('8', '8,2013-07-25 00:00:00.0,2911,PROCESSING')
('9', '9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT')
('10', '10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT')


In [37]:
# in the orderitem file make orderitem id and subtotal

orditem_s = orditem.map(lambda i : (i.split(',')[0],i.split(',')[4]))

print('The OrderItem id and Subtotal: ')
print('--------------------------------')
for s in orditem_s.take(10):
    print(s)

The OrderItem id and Subtotal: 
--------------------------------
('1', '299.98')
('2', '199.99')
('3', '250.0')
('4', '129.99')
('5', '49.98')
('6', '299.95')
('7', '150.0')
('8', '199.92')
('9', '299.98')
('10', '299.95')


In [38]:
def lower_case(s):
    return s.lower()

ord_l = ord.map(lambda l : lower_case(l.split(',')[3]))

print('The Lower case status:')
print('----------------------')
for l in ord_l.take(10):
    print(l)

The Lower case status:
----------------------
closed
pending_payment
complete
closed
complete
complete
complete
processing
pending_payment
pending_payment


In [41]:
# FlatMap in Low Level transformation

# word count in the order file

ord_wc = ord.flatMap(lambda wc : wc.split(',')).map(lambda c : (c,1)).reduceByKey(lambda x,y:x+y)

ord_wc.take(10)

[('1', 2),
 ('2013-07-25 00:00:00.0', 143),
 ('11599', 6),
 ('CLOSED', 7556),
 ('2', 5),
 ('256', 11),
 ('PENDING_PAYMENT', 15030),
 ('3', 8),
 ('12111', 7),
 ('COMPLETE', 22899)]

In [45]:
# filter in low level transformation

# to filter the records in 2013 which is complete or closed

ord_f = ord.filter(lambda f : (f.split(',')[3] in ['CLOSED','COMPLETE']) and (f.split(',')[1].split('-')[0] == '2013'))

ord_f.take(10)


                   


['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE',
 '6,2013-07-25 00:00:00.0,7130,COMPLETE',
 '7,2013-07-25 00:00:00.0,4530,COMPLETE',
 '12,2013-07-25 00:00:00.0,1837,CLOSED',
 '15,2013-07-25 00:00:00.0,2568,COMPLETE',
 '17,2013-07-25 00:00:00.0,2667,COMPLETE',
 '18,2013-07-25 00:00:00.0,1205,CLOSED']

In [55]:
# joins in transformation

# find the subtotal for each order and customer id

# to join the data we can make it into key and value pairs


ord_map = ord.map(lambda x : (x.split(',')[0],x.split(',')[2]))

res = f"order id and total ",ord_map.take(4)
print(res)

orditem_map = orditem.map(lambda x : (x.split(',')[1],x.split(',')[4]))

res1 = f"orderitem id and subtotal ",orditem_map.take(5)

print(res1)

#combine the two maps to join

ord_join = ord_map.join(orditem_map)

res3 = f"After Joining: ",ord_join.take(5)
print(res3)


('order id and total ', [('1', '11599'), ('2', '256'), ('3', '12111'), ('4', '8827')])
('orderitem id and subtotal ', [('1', '299.98'), ('2', '199.99'), ('2', '250.0'), ('2', '129.99'), ('4', '49.98')])


('After Joining: ', [('1', ('11599', '299.98')), ('4', ('8827', '49.98')), ('4', ('8827', '299.95')), ('4', ('8827', '150.0')), ('4', ('8827', '199.92'))])


In [60]:
# cogroup

# creating rdd for cogroup

sc = spark.sparkContext

x = sc.parallelize([('a',1),('b',3)])
y = sc.parallelize([('a',2)])

xy = x.cogroup(y)

#xy.first()

# to make view on non iterable object

for i,j in xy.take(2):
    print(i,str(map(list,j)))


b <map object at 0x7f2692f32650>
a <map object at 0x7f2692f32530>


In [62]:
# cartesian join ie. cross join

c = sc.parallelize([1,2,3])

res = c.cartesian(c)

res.take(



[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]

In [66]:
# total aggregations - count() and reduce()

# count the number of orders which are closed 

closed_ord = ord.filter(lambda x : x.split(',')[3] in ['CLOSED'])
closed_ord.count()

7556

In [74]:
# to find total quantity sold in order from 1 to 10
from operator import add
res = orditem.filter(lambda s : int(s.split(',')[10]) < 11).map(lambda s : int(s.split(',')[3]).reduce(add))

print(res)


PythonRDD[127] at RDD at PythonRDD.scala:53


In [80]:
# Key aggregations 

# groupByKey - to find each product aggreagate value

ordgrp = orditem.map(lambda z : (int(z.split(',')[2]),float(z.split(',')[4]))).groupByKey()

res = ordgrp.mapValues(sum).collect()

sorted(res)



[(19, 7999.35999999999),
 (24, 18477.689999999988),
 (35, 10399.34999999999),
 (37, 27327.190000000017),
 (44, 56330.60999999992),
 (58, 8699.709999999995),
 (60, 9999.9),
 (61, 8399.719999999996),
 (78, 18598.139999999992),
 (93, 20866.64999999999),
 (116, 44585.090000000026),
 (127, 8909.729999999996),
 (134, 20025.0),
 (135, 19426.0),
 (172, 27210.0),
 (191, 3667633.20000043),
 (203, 12799.679999999995),
 (208, 29999.85000000001),
 (216, 12096.0),
 (226, 5999.899999999999),
 (235, 29601.540000000005),
 (249, 46559.59000000002),
 (251, 19077.879999999994),
 (258, 19567.93999999999),
 (273, 22252.050000000003),
 (276, 29398.810000000012),
 (278, 36576.87000000003),
 (282, 27127.520000000033),
 (295, 18690.649999999994),
 (303, 13999.649999999994),
 (305, 13333.0),
 (306, 17638.039999999997),
 (311, 5937.299999999995),
 (359, 20597.939999999995),
 (364, 10799.639999999994),
 (365, 4421143.020001181),
 (403, 2891757.540001228),
 (502, 3147800.0),
 (564, 27210.0),
 (565, 67830.0),
 (567,

In [82]:
# reduceByKey

ex1 = sc.parallelize((('a',1),('b',3),('a',2),('c',2)))

ex1.reduceByKey(add).collect()

[('a', 3), ('b', 3), ('c', 2)]

In [89]:
# to find maximum revenue for each product in reduceByKey

ord_r = orditem.map(lambda z : (int(z.split(',')[2]),float(z.split(',')[4]))).reduceByKey(lambda x,y: x if x > y else y).take(5)
ord_r

[(957, 299.98), (1073, 199.99), (502, 250.0), (403, 129.99), (897, 124.95)]

In [95]:
# aggregateByKey

ord_data = sc.parallelize([
(1,'Mahesh',200),(2,'Pradeep',400),(3,'Nijan',300),(4,'Avinash',600),(5,'Sai',200)
])

#ord_data.take(4)

ordpair = ord_data.map(lambda x : (x[0],(x[1],x[2])))

ordpair.take(3)


[(1, ('Mahesh', 200)), (2, ('Pradeep', 400)), (3, ('Nijan', 300))]

In [96]:
# writing function for aggregateByKey function

zero_val = 0

def seqdata(acc,ele):
    if acc > ele[1]:
        return acc
    else:
        return ele[1]



def combdata(acc1,acc2):
    if acc1 > acc2:
        return acc1
    else:
        return acc2

aggord = ordpair.aggregateByKey(zero_val,seqdata,combdata)

for i in aggord.collect():
    print(i)

    
    

(1, 200)
(2, 400)
(3, 300)
(4, 600)
(5, 200)


In [99]:
# countByKey

# count no of orders in each status

ord_demo1 = ord.map(lambda x : (x.split(',')[3],1))

res = ord_demo1.countByKey()

res.items()


dict_items([('CLOSED', 7556), ('PENDING_PAYMENT', 15030), ('COMPLETE', 22899), ('PROCESSING', 8275), ('PAYMENT_REVIEW', 729), ('PENDING', 7610), ('ON_HOLD', 3798), ('CANCELED', 1428), ('SUSPECTED_FRAUD', 1558)])

In [101]:
# sorting in Transformations

# sort orders using customer id

exm1 = ord.map(lambda x : (int(x.split(',')[2]),x))

exm1.sortByKey(ascending=True).take(5)

[(1, '22945,2013-12-13 00:00:00.0,1,COMPLETE'),
 (2, '15192,2013-10-29 00:00:00.0,2,PENDING_PAYMENT'),
 (2, '33865,2014-02-18 00:00:00.0,2,COMPLETE'),
 (2, '57963,2013-08-02 00:00:00.0,2,ON_HOLD'),
 (2, '67863,2013-11-30 00:00:00.0,2,COMPLETE')]

In [103]:
# implementing rank ops using product dataset

prod = sc.textFile('RetailDB SalesData/Products/part-00000')

print('The Product data as follows: ')
print('------------------------------')
for p in prod.take(5):
    print(p)


The Product data as follows: 
------------------------------
1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy
2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat
3,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat
4,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat
5,2,Riddell Youth Revolution Speed Custom Footbal,,199.99,http://images.acmesports.sports/Riddell+Youth+Revolution+Speed+Custom+Football+Helmet


In [109]:
# to find top 5 products with the maximum price
prod = prod.filter(lambda s : s.split(',')[4] != '')
prod_pair = prod.map(lambda x : (float(x.split(',')[4]),x))

print('The Top 5 : ')
for p in prod_pair.take(5):
    print(p)

The Top 5 : 
(59.98, '1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy')
(129.99, "2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat")
(89.99, "3,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat")
(89.99, "4,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat")
(199.99, '5,2,Riddell Youth Revolution Speed Custom Footbal,,199.99,http://images.acmesports.sports/Riddell+Youth+Revolution+Speed+Custom+Football+Helmet')


In [111]:
# using takeOrdered Function

exm2 = sc.parallelize([10,5,23,9,-1,2])

exm2.takeOrdered(4)

[-1, 2, 5, 9]

In [113]:
# using product dataset

res = prod.takeOrdered(5,lambda k : -float(k.split(',')[4]))

res



['208,10,SOLE E35 Elliptical,,1999.99,http://images.acmesports.sports/SOLE+E35+Elliptical',
 '66,4,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill',
 '199,10,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill',
 '496,22,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill',
 '1048,47,"Spalding Beast 60"" Glass Portable Basketball ",,1099.99,http://images.acmesports.sports/Spalding+Beast+60%22+Glass+Portable+Basketball+Hoop']

In [119]:
# set Transformations

# union ops

jul_ord = ord.filter(lambda x : x.split(',')[1].split('-')[1] == '07').map(lambda x : x.split(',')[2])

aug_ord = ord.filter(lambda x : x.split(',')[1].split('-')[1] == '08').map(lambda x : x.split(',')[2])


res = jul_ord.union(aug_ord)

res1 = jul_ord.intersection(aug_ord)

print('Union: ',res.take(5))

print('Intersect: ',res1.take(5))




Union:  ['11599', '256', '12111', '8827', '11318']
Intersect:  ['12111', '11318', '7130', '5657', '9149']


In [130]:
# sampling 

exm4 = sc.parallelize(range(100),4)

res = exm4.sample(withReplacement = False,fraction=0.1,seed=5).collect()
res1 = exm4.takeSample(withReplacement = True,num=10,seed=5)
print('Sample:',res)
print('Take Sample:',res1)



Sample: [10, 15, 28, 35, 50, 54, 59, 61, 73, 89, 93]
Take Sample: [72, 98, 97, 10, 44, 76, 18, 44, 21, 95]


In [132]:
# partitions

df = spark.range(10000)

df = df.select(df.id,df.id*2,df.id*3)

df = df.union(df)
df = df.union(df)
df = df.union(df)
df = df.union(df)
df = df.union(df)

p_df = df.rdd.map(lambda x : str(x[0])+','+str(x[1])+','+str(x[2]))

p_df.coalesce(1).saveAsTextFile('data')



In [ ]:
# to save the files

